In [ ]:
# Homework no. 1 for Computer Vision course with Bogdan Alexe
Computer vision
Automatic grading of single choice tests

- The first thing we have to do is to take only the part of the image that is needed for grading (the part with the two tables). 
- After we have taken the part with the tables, we have to split the image in the left part (MATEMATICA) and the right part (INFORMATICA/FIZICA).
right_image.png

Having the image from above, we have to find the lines and the columns that contain the student's answers.
image.png

After we have found the tables, we have to crop the tables in order to take only the student's answers.
table.png

A student can choose between INFORMATICA/FIZICA, our goal is to find the 2 boxes representing the number of the subject.


In [107]:
# imports
import cv2 as cv
import numpy as np
import glob
import os

In [108]:
# Taken from lab no. 2
# read the name of the images
base_folder = '../1st Project/Files/images'
images_names = glob.glob(os.path.join(base_folder, "image_*.jpg")) 
idx_image = -1
SHOW_INTERMEDIATE_RESULTS = True # set this to false in order to show only the final result
NUM_OF_SECONDS = 0 # set to 0 in order to wait until keypress

In [109]:
idx_image = 1

In [129]:
# read the idx-th image
image = cv.imread(images_names[idx_image])
#make it 0.15 times on every dimension
img_resize = cv.resize(image, (0, 0), fx=0.15, fy=0.15)
print(image.shape)
print(img_resize.shape)
if SHOW_INTERMEDIATE_RESULTS:
    cv.imshow("image", img_resize)
    cv.waitKey(NUM_OF_SECONDS)
    cv.destroyAllWindows()

(5847, 4132, 3)
(877, 620, 3)


In [221]:
# take the lower part of the image
image_h, image_w, channels = img_resize.shape
print(img_resize.shape)
# DIMENSIONS ARE SET MANUALLY. REVISE THIS LATER.
# an idea is to search for the corners after the 2nd part ofthe image
# img_crop = img_resize[400:orig_h - 100, 0:orig_w]
img_crop = img_resize[int(0.10*orig_h):int(orig_h*0.90),:]
if SHOW_INTERMEDIATE_RESULTS:
    cv.imshow("image", img_crop)
    cv.waitKey(NUM_OF_SECONDS)
    cv.destroyAllWindows()

(877, 620, 3)


In [222]:
#make it grayscale
img_gray = cv.cvtColor(img_crop, cv.COLOR_BGR2GRAY)
if SHOW_INTERMEDIATE_RESULTS:
    cv.imshow("image", img_gray)
    cv.waitKey(NUM_OF_SECONDS)
    cv.destroyAllWindows()

In [242]:
#obtain a binary image by applying an adaptive threshold = mean value of img1_gray
meanValue = img_gray.mean()
print(meanValue)
_, img_th = cv.threshold(img_gray, meanValue, 255, cv.THRESH_BINARY_INV) # the second params is the threshold
cv.imshow("image", img_th)
cv.waitKey(NUM_OF_SECONDS)
cv.destroyAllWindows()

238.51802453818584


In [243]:
#define a vertical kernel and filter the binary image
kernel = np.array([[0,1,0],[0,1,0],[0,1,0]]) 

print(kernel.sum())
kernel = kernel / kernel.sum()
print(kernel)
print(kernel.sum())

3
[[0.         0.33333333 0.        ]
 [0.         0.33333333 0.        ]
 [0.         0.33333333 0.        ]]
1.0


In [244]:
thresholded = img_th
filtered = 255 - cv.filter2D(255 - thresholded, -1, kernel)
filtered[filtered < 255] = 0
filtered_vertical = filtered

In [245]:
if SHOW_INTERMEDIATE_RESULTS:
    cv.imshow("image", filtered_vertical)
    cv.waitKey(NUM_OF_SECONDS)
    cv.destroyAllWindows()

In [257]:
#define a horizontal kernel and filter the binary image
kernel = np.array([11 * [0], 11 * [1], 11 * [0]])                
print(kernel.sum())
kernel = kernel / kernel.sum()
print(kernel)
print(kernel.sum())

110
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.09090909 0.09090909 0.09090909 0.09090909 0.09090909 0.09090909
  0.09090909 0.09090909 0.09090909 0.09090909 0.09090909]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]]
1.0


In [258]:
thresholded = img_th
filtered = 255 - cv.filter2D(255 - thresholded, -1, kernel)
filtered[filtered < 255] = 0
filtered_horizontal = filtered

In [259]:
if SHOW_INTERMEDIATE_RESULTS:
    cv.imshow("image", filtered_horizontal)
    cv.waitKey(NUM_OF_SECONDS)
    cv.destroyAllWindows()

In [249]:
# find horizontal lines
mask = filtered_horizontal // 255
print(mask.min())
print(mask.max())
res = np.sum(mask, axis=1) 
# print(res.shape)
# print(res)
res = res.argsort() 
# print(res)

0
1


In [250]:
num_lines = 60
h_img = np.dstack((filtered_horizontal, filtered_horizontal, filtered_horizontal))
lines = [] #  _ x 
for i in range(num_lines+1):
    cv.line(h_img, (0, res[-i]), (h_img.shape[1], res[-i]), (0, 0, 255), 2) 
    lines.append([(0, res[-i]), (h_img.shape[1], res[-i])])
        
if SHOW_INTERMEDIATE_RESULTS:
    cv.imshow("h_img", h_img)
    cv.waitKey(NUM_OF_SECONDS)
    cv.destroyAllWindows()

In [256]:
# sort only on y0 !!! only
lines.sort(key=lambda coords: coords[0][1])
    
threshold_same_line = 11
distinct_lines = []   
distinct_lines.append(lines[0])
     
for line in lines:  
    if line[0][1] - distinct_lines[-1][0][1] > threshold_same_line:
        distinct_lines.append(line)   
    
# take the last 16 lines
correct_lines = distinct_lines[-16:] 
color_image = np.dstack((img_gray, img_gray, img_gray))
for line in correct_lines: 
    cv.line(color_image, line[0], line[1], (255, 0, 0), 1) 
        
if SHOW_INTERMEDIATE_RESULTS:
    cv.imshow("Detected lines", color_image)
    cv.waitKey(NUM_OF_SECONDS)
    cv.destroyAllWindows()